<a href="https://colab.research.google.com/github/ConorD28/Assessments/blob/main/WS_Research_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
df_ = pd.read_csv('World Series Research_Upload.csv')
df2 = df_.iloc[:, 1:992] # Select columns to train
df3 = df2.replace('-', np.NaN)
df4 = df3.dropna(axis='columns')

In [5]:
def correlation(dataset, threshold, target): #Function to get Pearson's correlation between input and target
  data = []
  for i in range(len(dataset.columns)):
      cor2 = dataset.iloc[:,i].corr(target)
      column_headers = list(dataset.columns.values)
      if(abs(cor2) > threshold):
        data.append(dataset.iloc[:,i])
      i = i + 1
  return data

In [6]:
from sklearn.model_selection import train_test_split
from numpy.random.mtrand import random_sample
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [7]:
def Scores(y, y_pred, y_full):
  MSE = mean_squared_error(y, y_pred)
  MAE = mean_absolute_error(y, y_pred)
  Normalized_RMSE = (np.sqrt(MSE)/np.mean(y_full))*100
  Normalized_MAE = (MAE/np.mean(y_full))*100
  Avg_Normalized_Score = (Normalized_RMSE + Normalized_MAE)/2
  print(f'Avg. Normalized Score:{ Avg_Normalized_Score:.1f}%')
  print(f'Normalized RMSE:{ Normalized_RMSE:.1f}%')
  print(f'Normalized MAE:{ Normalized_MAE:.2f}%')
  print(f'MAE:{ MAE:.3f}')
  print(f'RMSE:{ np.sqrt(MSE):.3f}')
  return Avg_Normalized_Score

In [8]:
def RLE_Model(X, y, choice, test): #Function to run Ridge, Lasso, or ElasticNet model
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0) #Train/Test

  if(choice=="Ridge"):
    alphas = np.geomspace(1e-10, 1e10, num=100)
    pipeline = make_pipeline(StandardScaler(), RidgeCV(alphas=alphas))
    pipeline.fit(X_train, y_train)

  if(choice=="Lasso"):
    alphas = np.geomspace(1e-10, 1e10, num=21)
    pipeline = make_pipeline(StandardScaler(), LassoCV(alphas=alphas))
    pipeline.fit(X_train, y_train)

  if(choice=="Elastic"):
    l1_ratio = [0, 0.3, 0.5, 0.7,  0.9, 1]
    alphas = np.geomspace(1e-10, 1e10, num=100)
    pipeline = make_pipeline(StandardScaler(), ElasticNetCV(alphas=alphas, l1_ratio=l1_ratio, max_iter=100000))
    pipeline.fit(X_train, y_train)

  print(f'Chosen alpha  {pipeline.steps[1][1].alpha_:.6f}')
  print(f'Intercept (b) {pipeline.steps[1][1].intercept_:.6f}')
  #print(pd.Series(pipeline.steps[1][1].coef_, index=X.columns),'\n')

  #Calculate the predicted values:
  y_train_pred = pipeline.predict(X_train)
  print(y_train_pred)
  print()

  y_test_pred = pipeline.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  print()

  #Predictions:
  print(y_test_pred)
  print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  predictions = pipeline.predict(test)
  print(predictions)

In [ ]:
#Do the following to get autosklearn to work in Google Colab:
!pip install Cython==0.29.36
!pip uninstall scipy -y
!pip install scipy==1.9
!pip uninstall pyparsing -y
!pip install pyparsing==2.4
!pip uninstall scikit_learn -y
!pip uninstall imbalanced-learn -y
!pip uninstall mlxtend -y
!pip uninstall yellowbrick -y
!pip install scikit-learn==0.24.2 --no-build-isolation
!pip install auto-sklearn

In [ ]:
#Do the following to get autosklearn to work in Google Colab:
!pip uninstall scipy -y
!pip install scipy==1.9
!pip uninstall pyparsing -y
!pip install pyparsing==2.4
!pip uninstall scikit_learn -y
!pip install scikit-learn==0.24.2 --no-build-isolation

In [ ]:
#Do the following to get autosklearn to work in Google Colab:
!pip uninstall pyparsing -y
!pip install pyparsing==2.4

In [1]:
import autosklearn

In [2]:
from pprint import pprint
import sklearn.datasets
import sklearn.metrics
import autosklearn.regression
import matplotlib.pyplot as plt

def Auto_Model(X, y, test): #Uses AutoML to make a model
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0) #Train/Test

  automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder="/tmp/autosklearn_regression_example_tmp",
  )
  automl.fit(X_train, y_train, dataset_name="WS")

  test_predictions = automl.predict(X_test)
  y_train_pred = automl.predict(X_train)

  print(automl.leaderboard())
  print()

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  print()

  #Predictions:
  print(test_predictions)
  print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, test_predictions, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  predictions2 = automl.predict(test)
  print(predictions2)

In [9]:
#Get Correlated Variables:
data_R_GM = correlation(df4, .515, df_["RPerGM_PLBT"])
df_imp_variables_R_GM = pd.DataFrame(data_R_GM)
df_imp_variables_R_GM2 = df_imp_variables_R_GM.transpose()

data_R_PA = correlation(df4, .498, df_["RPerPA_PLBT"])
df_imp_variables_R_PA = pd.DataFrame(data_R_PA)
df_imp_variables_R_PA2 = df_imp_variables_R_PA.transpose()

df_PD = df4.iloc[:, 323:992] #Pitching and Defense stats
data_RA_GM = correlation(df_PD, .52, df_["RAPerGM_PLPT"])
df_imp_variables_RA_GM3 = pd.DataFrame(data_RA_GM)
df_imp_variables_RA_GM4 = df_imp_variables_RA_GM3.transpose()

data_RA_Batter = correlation(df4, .528, df_["RAPerBatter_PLPT"])
df_imp_variables_RA_Batter = pd.DataFrame(data_RA_Batter)
df_imp_variables_RA_Batter2 = df_imp_variables_RA_Batter.transpose()

In [10]:
#RAGM
df_predict = pd.read_csv("2023 MLB Playoff Team Stats_Upload_RAGM.csv") #This year's playoff teams' stats
df_predict2 = df_predict.iloc[:, 0:48]
df_predict3 = df_predict2.reindex(sorted(df_predict2.columns), axis=1) #get variables in alphabetical order
#so they are in the same order as the model's variables

df_imp_variables_RA_GM5 = df_imp_variables_RA_GM4.reindex(sorted(df_imp_variables_RA_GM4.columns), axis=1) #get variables in alphabetical order

XPD = df_imp_variables_RA_GM5
yPD = df_["RAPerGM_PLPT"] #RAPerGM_PLPT
RLE_Model(XPD, yPD, "Elastic", df_predict3) #Avg. Score:5.2%, Difference of avg scores:0.769%, Normalized RMSE:5.3%, Normalized MAE:5.04%

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.1113651368930832, tolerance: 0.0006222730288810908
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.1113651324436367, tolerance: 0.0006222730288810908
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.111365125358855, tolerance: 0.0006222730288810908
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarn

Chosen alpha  0.048626
Intercept (b) 3.463891
[3.9773736  2.87112679 3.38504235 3.42280894 2.86022452 4.31583378
 3.58071425 3.58061872 2.67608415 2.96347938 4.18448751 2.62458269
 4.9203737  3.13171762]

Avg. Normalized Score:4.4%
Normalized RMSE:4.8%
Normalized MAE:4.03%
MAE:0.140
RMSE:0.166

[3.74101138 3.25070583 4.0573374  3.20810325 3.38799476 3.58162025]

Avg. Normalized Score:5.2%
Normalized RMSE:5.3%
Normalized MAE:5.04%
MAE:0.175
RMSE:0.185
Difference of avg scores:0.769%

[3.52831934 3.52962924 3.46014481 3.66152689 3.4841854  3.78067087
 3.65599022 3.52849841 3.65116556 3.71734626 4.70409914 3.41083904]


In [12]:
#RABat
df_predict = pd.read_csv("2023 Team Stats_RABat.csv") #This year's playoff teams' stats
df_predict2 = df_predict.reindex(sorted(df_predict.columns), axis=1)

df_imp_variables_RA_Bat_ = df_imp_variables_RA_Batter2.reindex(sorted(df_imp_variables_RA_Batter2.columns), axis=1)

X4 = df_imp_variables_RA_Bat_
y4 = df_["RAPerBatter_PLPT"]
Auto_Model(X4,y4, df_predict2) #Avg. Normalized Score:8.3%, Difference of avg scores:3.583%, Normalized RMSE:9.1%, Normalized MAE:7.54%
#RLE_Model(X4,y4, "Ridge")

          rank  ensemble_weight              type      cost  duration
model_id                                                             
6            1             0.56  gaussian_process  0.643868  0.987083
34           2             0.36        libsvm_svr  0.937134  0.902402
29           3             0.06    ard_regression  1.070589  1.183038
23           4             0.02  gaussian_process  1.532413  0.813839

Avg. Normalized Score:4.8%
Normalized RMSE:5.2%
Normalized MAE:4.32%
MAE:0.004
RMSE:0.005

[0.10288019 0.08511726 0.10215217 0.08375862 0.09229917 0.09941684]

Avg. Normalized Score:8.3%
Normalized RMSE:9.1%
Normalized MAE:7.54%
MAE:0.007
RMSE:0.009
Difference of avg scores:3.583%

[0.1017065  0.0958386  0.09255528 0.10223557 0.10010798 0.10025542
 0.09903677 0.09399664 0.09732138 0.09425576 0.10054684 0.09861501]


In [13]:
#R/GM
df_predictRGM = pd.read_csv("2023 MLB Playoff Team Stats_Upload_RGM_40.csv") #This year's playoff teams' stats
df_predictRGM2 = df_predictRGM.reindex(sorted(df_predictRGM.columns), axis=1)

df_imp_variables_R_GM3 = df_imp_variables_R_GM2.reindex(sorted(df_imp_variables_R_GM2.columns), axis=1)

X1 = df_imp_variables_R_GM3
y1 = df_["RPerGM_PLBT"]
RLE_Model(X1,y1, "Lasso", df_predictRGM2) #Lasso:, Avg. Score:8.3%, Difference of avg scores:4.053%, Normalized RMSE:10.0%, Normalized MAE:6.63%
#Auto_Model(X1,y1, df_predict3RGM)

Chosen alpha  0.100000
Intercept (b) 4.397753
[5.2713767  3.65101688 4.41298781 4.1863124  3.88060018 5.48465274
 4.57451661 4.08694583 4.02154012 3.52080735 5.35500937 3.95384168
 4.81608153 4.3528458 ]

Avg. Normalized Score:4.2%
Normalized RMSE:4.7%
Normalized MAE:3.75%
MAE:0.169
RMSE:0.213

[5.28343823 4.08542495 4.46058117 4.14076914 4.60505411 4.70275569]

Avg. Normalized Score:8.3%
Normalized RMSE:10.0%
Normalized MAE:6.63%
MAE:0.298
RMSE:0.448
Difference of avg scores:4.053%

[4.04831287 4.35425602 4.53656028 4.62007974 4.00858553 4.25492852
 4.45823107 3.79909859 4.45494196 3.36610193 4.51442036 4.19085321]


In [14]:
#R/PA
df_predict = pd.read_csv("2023 MLB Team Stats_Upload_RPA.csv") #This year's playoff teams' stats
df_predict2 = df_predict.iloc[:, 0:52]
df_predict3 = df_predict2.reindex(sorted(df_predict2.columns), axis=1)

df_imp_variables_R_PA3 = df_imp_variables_R_PA2.reindex(sorted(df_imp_variables_R_PA2.columns), axis=1)

X2 = df_imp_variables_R_PA3
y2 = df_["RPerPA_PLBT"]
Auto_Model(X2,y2, df_predict3) #Avg. Normalized Score:9.3%, Difference of avg scores:5.049%, Normalized RMSE:10.3%, Normalized MAE:8.25%
#RLE_Model(X2,y2, "Ridge", df_predict3)

          rank  ensemble_weight              type      cost  duration
model_id                                                             
3            1              0.7  gaussian_process  0.776778  1.196287
40           2              0.3    ard_regression  1.309930  0.721672

Avg. Normalized Score:4.2%
Normalized RMSE:4.8%
Normalized MAE:3.56%
MAE:0.004
RMSE:0.006

[0.12869247 0.10248854 0.12570243 0.10891728 0.11615601 0.12700336]

Avg. Normalized Score:9.3%
Normalized RMSE:10.3%
Normalized MAE:8.25%
MAE:0.010
RMSE:0.012
Difference of avg scores:5.049%

[0.12148621 0.11576276 0.11807354 0.12088706 0.11761153 0.11596447
 0.11662273 0.11370528 0.11626359 0.11335802 0.12177106 0.11576001]
